In [14]:
# Importing libraries
import pandas as pd
from datetime import datetime

In [15]:
# Reading the CSV file
file_path = "Crime_Data_from_2020_to_Present.csv" 
df = pd.read_csv(file_path)

In [16]:
sample_df=df.sample(n=1000)

In [17]:
df.shape

(938457, 28)

In [18]:
# Checking for Null Values
df.isnull().sum()

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           131889
Vict Age               0
Vict Sex          125454
Vict Descent      125464
Premis Cd             10
Premis Desc          563
Weapon Used Cd    615391
Weapon Desc       615391
Status                 0
Status Desc            0
Crm Cd 1              11
Crm Cd 2          870521
Crm Cd 3          936181
Crm Cd 4          938392
LOCATION               0
Cross Street      791589
LAT                    0
LON                    0
dtype: int64

In [19]:
# Date format correction
def correct_and_extract_date(date_string):
    try:
        # Try parsing the date string
        date = datetime.strptime(date_string, '%m/%d/%y %H:%M')
        return date.strftime('%m/%d/%Y')
    except ValueError:
        try:
            # Try parsing the date string with a different format
            date = datetime.strptime(date_string, '%m/%d/%Y %I:%M:%S %p')
            return date.strftime('%m/%d/%Y')
        except ValueError:
            return "Invalid Date Format"

def rep(lit):
    return lit.replace("  ", " ")

def ledzero(lit):
    val = str(lit).zfill(4)
    return val[:2] + ":" + val[2:]

In [20]:

# Data preprocessing for DateTime fields
df['Date Rptd'] = df['Date Rptd'].apply(correct_and_extract_date)
df['DATE OCC'] = df['DATE OCC'].apply(correct_and_extract_date)
df['TIME OCC'] = df["TIME OCC"].apply(ledzero)
df['TIME OCC'] = df["TIME OCC"]

df['DATE OCC'] = df["DATE OCC"] + " " + df["TIME OCC"]


In [21]:
# Trimming all the spaces in LOCATION field
for _ in range(0, 5):
    df['LOCATION'] = df['LOCATION'].apply(rep)

# run this 5-7 times to remove all the spaces


In [22]:
# Cleanig the rows that have (0,0) coordinates
def remove_zero_coordinates(df):
    # Filter out rows where both latitude and longitude are 0
    filtered_df = df[(df['LAT'] != 0) | (df['LON'] != 0)]
    return filtered_df

In [23]:
df = remove_zero_coordinates(df)

In [24]:
# We have data until 2024 April, but untill 2023 Dec is suffice for visualization
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])

# Extract year from 'DATE OCC' column
df['Year'] = df['DATE OCC'].dt.year


# Group by year and count the number of rows
yearly_counts = df.groupby('Year').size()

# Print the result
print("Number of rows in each year:")
print(yearly_counts)
df["Year"].drop(index=1)

Number of rows in each year:
Year
2020    198894
2021    208222
2022    234939
2023    231526
2024     62612
dtype: int64


0         2020
2         2020
3         2020
4         2020
5         2020
          ... 
938452    2024
938453    2024
938454    2024
938455    2024
938456    2024
Name: Year, Length: 936192, dtype: int32

In [25]:
# Trimming the dataset until 2023
df = df[df['DATE OCC'].dt.year <= 2023]

In [26]:
# Generating the clean dataset from dataframe
output_file_path = "Crime_Data_2020-2023.csv" 
df.to_csv(output_file_path, index=False)

# Print confirmation message
print("DataFrame saved to:", output_file_path)

DataFrame saved to: Crime_Data_2020-2023.csv
